In [30]:
from pyspark.sql import SparkSession
from pyspark.ml import image

In [2]:
# Create a SparkSession
spark = (SparkSession
      .builder
      .appName("SparkSQLExampleApp")
      .getOrCreate())

22/04/20 17:55:48 WARN Utils: Your hostname, Zipcoders-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.77 instead (on interface en0)
22/04/20 17:55:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/20 17:55:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Path to data set
csv_file = "/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

In [4]:
df = (spark.read.format("csv")
      .option("inferSchema", "true")
      .option("header", "true")
      .load(csv_file))

In [5]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [6]:
# In Python
schema = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"

In [7]:
spark.sql("""SELECT distance, origin, destination
    FROM us_delay_flights_tbl WHERE distance > 1000
    ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [8]:
spark.sql("""SELECT date, delay, origin, destination
    FROM us_delay_flights_tbl
    WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
    ORDER by delay DESC""").show(10)


+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [9]:
spark.sql("""SELECT delay, origin, destination,
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
                  WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                  WHEN delay > 0 and delay < 60  THEN  'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'Early'
               END AS Flight_Delays
               FROM us_delay_flights_tbl
               ORDER BY origin, delay DESC""").show(10)


+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [10]:
from pyspark.sql.functions import col, desc

All three of the preceding SQL queries can be expressed with an equivalent Data‐ Frame API query. For example, the first query can be expressed in the Python Data‐ Frame API as:

In [11]:
(df.select("distance", "origin", "destination")
      .where("distance > 1000")
      .orderBy("distance", ascending=False).show(10))

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



# Creating SQL Databases and Tables

In [12]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

To create a managed table within the database learn_spark_db, you can issue a SQL query like the following:

In [13]:
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

22/04/20 17:56:02 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `learn_spark_db`.`managed_us_delay_flights_tbl`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


In [14]:
# You can do the same thing using the DataFrame API like this:
# Path to our US flight delays CSV file
csv_file = "/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv" # Schema as defined in the preceding example
schema="date STRING, delay INT, distance INT, origin STRING, destination STRING" 
flights_df = spark.read.csv(csv_file, schema=schema) 
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")    

AnalysisException: Can not create the managed table('`managed_us_delay_flights_tbl`'). The associated location('file:/Users/allenc/PyCharmProjects/LearningSparkV2/notebooks/spark-warehouse/learn_spark_db.db/managed_us_delay_flights_tbl') already exists.

In [15]:
# Creating an unmanaged table

In [16]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT,
    distance INT, origin STRING, destination STRING)
    USING csv OPTIONS (PATH
    'Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')""")

22/04/20 18:06:46 WARN HadoopFSUtils: The directory file:/Users/allenc/PyCharmProjects/LearningSparkV2/notebooks/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv was not found. Was it deleted very recently?


DataFrame[]

In [17]:
# Creating Views 
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
# Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [18]:
spark.read.table("us_origin_airport_JFK_tmp_view")

DataFrame[date: int, delay: int, origin: string, destination: string]

In [19]:
# Reading a JSON file into a Spark SQL table

In [20]:
file = "/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/summary-data/json/2010-summary.json"
df = spark.read.format("json").load(file)

In [21]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [22]:
file = "/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
df = (spark.read.format("csv")
    .option("header", "true")
    .schema(schema)
    .option("mode", "FAILFAST") # Exit if any errors 
    .option("nullValue", "") # Replace any null data field with quotes 
    .load(file))

In [23]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

# Writing DataFrames to CSV files

In [24]:
# df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

In [29]:
# Reading an Avro file into a DataFrame
df = (spark.read.format("avro")
    .load("/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*/*"))
df.show(truncate=False)

AnalysisException:  Failed to find data source: avro. Avro is built-in but external data source module since Spark 2.4. Please deploy the application as per the deployment section of "Apache Avro Data Source Guide".        

In [31]:
# Reading an image file into a DataFrame

In [32]:
image_dir = "/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/cctvVideos/train_images"
images_df = spark.read.format("image").load(image_dir)
images_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)
 |-- label: integer (nullable = true)



In [33]:
images_df.select("image.height", "image.width", "image.nChannels", "image.mode",
"label").show(5, truncate=False)

+------+-----+---------+----+-----+
|height|width|nChannels|mode|label|
+------+-----+---------+----+-----+
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |1    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
|288   |384  |3        |16  |0    |
+------+-----+---------+----+-----+
only showing top 5 rows



In [34]:
# Reading a binary file into a DataFrame

In [35]:
path = "/Users/allenc/PyCharmProjects/LearningSparkV2/databricks-datasets/learning-spark-v2/cctvVideos/train_images"
binary_files_df = (spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .load(path))
binary_files_df.show(5)

+--------------------+--------------------+------+--------------------+-----+
|                path|    modificationTime|length|             content|label|
+--------------------+--------------------+------+--------------------+-----+
|file:/Users/allen...|2022-04-19 11:27:...| 55037|[FF D8 FF E0 00 1...|    0|
|file:/Users/allen...|2022-04-19 11:27:...| 54634|[FF D8 FF E0 00 1...|    1|
|file:/Users/allen...|2022-04-19 11:27:...| 54624|[FF D8 FF E0 00 1...|    0|
|file:/Users/allen...|2022-04-19 11:27:...| 54505|[FF D8 FF E0 00 1...|    0|
|file:/Users/allen...|2022-04-19 11:27:...| 54475|[FF D8 FF E0 00 1...|    0|
+--------------------+--------------------+------+--------------------+-----+
only showing top 5 rows



In [37]:
# To ignore partitioning data discovery in a directory, you can set recursiveFile Lookup to "true":
binary_files_df = (spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.jpg")
    .option("recursiveFileLookup", "true")
    .load(path))
binary_files_df.show(5)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/Users/allen...|2022-04-19 11:27:...| 55037|[FF D8 FF E0 00 1...|
|file:/Users/allen...|2022-04-19 11:27:...| 54634|[FF D8 FF E0 00 1...|
|file:/Users/allen...|2022-04-19 11:27:...| 54624|[FF D8 FF E0 00 1...|
|file:/Users/allen...|2022-04-19 11:27:...| 54505|[FF D8 FF E0 00 1...|
|file:/Users/allen...|2022-04-19 11:27:...| 54475|[FF D8 FF E0 00 1...|
+--------------------+--------------------+------+--------------------+
only showing top 5 rows



# Summary

To recap, this chapter explored the interoperability between the DataFrame API and Spark SQL. In particular, you got a flavor of how to use Spark SQL to:
• Create managed and unmanaged tables using Spark SQL and the DataFrame API.
• Read from and write to various built-in data sources and file formats.
• Employ the spark.sql programmatic interface to issue SQL queries on struc‐ tured data stored as Spark SQL tables or views.
• Peruse the Spark Catalog to inspect metadata associated with tables and views.